In [2]:

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
from scipy.linalg import expm
import pickle
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_step_heisenberg_qt, ham_evol_qt

np.random.seed(667)

In [3]:
num_qubits = 3
num_energy_bits = 6
delta = 0.1 
eps = 0.1
sigma = 5
bohr_bound = 2 ** (-num_energy_bits + 1)
beta = 1
eig_index = 2
liouv_time = 1

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)

initial_state = hamiltonian.eigenstates[:, eig_index]
initial_dm = DensityMatrix(initial_state).data

Original spectrum:  [-3.7    -3.177  -2.5482 -1.7     1.625   2.4547  3.2521  3.7936]
Ideal spectrum:  [0.     0.0314 0.0692 0.1201 0.3198 0.3696 0.4175 0.45  ]
Nonrescaled coefficients:  [1.  0.8 0.9 1. ]
Rescaled coefficients:  [0.06005156 0.04804125 0.05404641 0.06005156]


In [4]:
# Create Gibbs state from eigenstates and eigenenergies
gibbs = expm(-beta * hamiltonian.qt.full()) / np.trace(expm(-beta * hamiltonian.qt.full()))
print(gibbs.shape)

(8, 8)


In [8]:
# Liouvillian, could be from (1.8), i.e. integral over the energies

N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
energy_labels = 2 * np.pi * N_labels / N 
time_labels = N_labels

jump_op = qt.tensor([qt.qeye(2), qt.sigmax(), qt.qeye(2)]).full()

boltzmann = lambda beta, energy: np.min([1, np.exp(-beta * energy)])
#?How do I get Liouvillian without a reference DM?

# liouvillian = np.zeros((2**num_qubits, 2**num_qubits), dtype=complex)
# for phase in N_labels:
#     energy = 2 * np.pi * phase / 2**num_energy_bits
#     oft_op = oft(jump_op, phase, 2**num_energy_bits, sigma, hamiltonian=hamiltonian.qt.full())
#     liouvillian += boltzmann(beta, energy)*(-0.5 * oft_op.conj().T @ oft_op @ initial_dm
#                                       -0.5 * initial_dm @ oft_op.conj().T @ oft_op
#                                       + oft_op @ initial_dm @ oft_op.conj().T)
    
    
    
liouvillian_superop = np.zeros((N**2, N**2), dtype=complex)
for phase in N_labels:
    energy = 2 * np.pi * phase / 2**num_energy_bits
    oft_op = qt.Qobj(np.sqrt(boltzmann(beta, energy)) * oft(jump_op, phase, N, sigma, hamiltonian=hamiltonian.qt.full()))
    liouvillian_superop = qt.sprepost(oft_op, oft_op.dag()) - 0.5 * qt.sprepost(oft_op.dag() * oft_op, qt.qeye(N)) \
        - 0.5 * qt.sprepost(qt.qeye(N), oft_op.dag() * oft_op)



TypeError: Incompatible quantum object dimensions